In [32]:
import os
import sys
from Flight_Fare.exception import CustomException
from Flight_Fare.logger import logging
from Flight_Fare.config.configuration import Configuration
from Flight_Fare.entity.artifact_entity import *
from Flight_Fare.entity.config_entity import *
from Flight_Fare.constant import *
from Flight_Fare.util.util import *
from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pandas as pd 

In [133]:
train_df = pd.read_csv(r"Flight_Fare\artifact\data_ingestion\2023-07-25-01-27-30\ingested_data\train\Flight_Fare_Prediction.csv")
test_df = pd.read_csv(r"Flight_Fare\artifact\data_ingestion\2023-07-25-01-27-30\ingested_data\test\Flight_Fare_Prediction.csv")

In [158]:
class FeatureGenerator(BaseEstimator,TransformerMixin):

    def __init__(self,columns):
        self.columns = columns

    def fit(self,X,y=None):
        return X

    def transform(self,X,y=None):

        if self.columns == 'Date_of_Journey':
            try:
                X['journey_day'] = X['Date_of_Journey'].str.split('/').str[0].astype(int)
                X['journey_month'] = X['Date_of_Journey'].str.split('/').str[1].str[1].astype(int)

                # Since we have converted Date_of_Journey column into integers, Now we can drop as it is of no use.
                X.drop('Date_of_Journey',inplace = True,axis = 1)
            except Exception as e:
                raise CustomException(sys,e)
            
        elif self.columns == 'Dep_Time':
            try:
                X['Dep_hr'] = X['Dep_Time'].str.split(':').str[0].astype(int)
                X['Dep_min'] = X['Dep_Time'].str.split(':').str[1].astype(int)

                # Since we have converted Dep_Time column into integers, Now we can drop as it is of no use.
                X.drop('Dep_Time',inplace = True,axis = 1)

            except Exception as e:
                raise CustomException(sys,e)
            

        elif self.columns == 'Arrival_Time':
            try:
                X['Arrival_hr'] = X['Arrival_Time'].str.split(" ").str[0].str.split(':').str[0].astype(int)
                X['Arrival_min'] = X['Arrival_Time'].str.split(" ").str[0].str.split(':').str[1].astype(int)


                # Since we have converted Dep_Time column into integers, Now we can drop as it is of no use.
                X.drop('Arrival_Time',inplace = True,axis = 1)

            except Exception as e:
                raise CustomException(sys,e)
            

        elif self.columns == 'Duration':
            X['Duration_hr'] = 0
            X['Duration_min'] = 0
        
            try:

                duration = list(X['Duration'])

                for i in range(len(duration)):
                    if 'h' not in duration[i]:
                        duration[i] = '0h ' + duration[i]
                        
                    if 'm' not in duration[i]:
                        duration[i] = duration[i] + ' 0m'
                        
                for i in range(len(duration)):
                    hr = int(duration[i].split("h")[0])
                    min_ = int(duration[i].split(" ")[1].split("m")[0])
                    X['Duration_hr'][i] = hr
                    X['Duration_min'][i] = min_
                    
                    
                # Since we have converted Duration column into integers, Now we can drop as it is of no use.

                X.drop('Duration',inplace = True,axis = 1)


                # Since there are some nan values represting 0 mins , we are replacing those values with 0 

                X['Duration_min'].fillna(0,inplace = True)

            except Exception as e:
                raise CustomException(sys,e)
            
        return X

In [159]:
class DataTransformation:
    def __init__(self,data_transformation_config: DataTransformationConfig,
                 data_ingestion_artifact: DataIngestionArtifact,
                 data_validation_artifact: DataValidationArtifact):
        
        try:
            logging.info(f"{'>>' * 30}Data Transformation log started.{'<<' * 30} ")
            self.data_transformation_config= data_transformation_config
            self.data_ingestion_artifact = data_ingestion_artifact
            self.data_validation_artifact = data_validation_artifact

        except Exception as e:
            raise CustomException(e,sys) from e
        

    def get_data_transformer_object(self) -> ColumnTransformer:
        try:
            num_col = ['Date_of_Journey','Dep_Time','Arrival_Time','Duration']

            num_pipeline = Pipeline(
                steps =[
                    ("Featur_genrator",FeatureGenerator(num_col)),
                    ("scalar",StandardScaler())
                ]
            )

            preprocesser = ColumnTransformer(
                [
                ("num_pipeline",num_pipeline,num_col)
                ]
            )

            return preprocesser
        
        except Exception as e:
            raise CustomException(e,sys) from e

In [160]:
trans = DataTransformation(data_transformation_config =  DataTransformationConfig,
                             data_ingestion_artifact =  DataIngestionArtifact,
                             data_validation_artifact = DataValidationArtifact)
preprocessing_obj = trans.get_data_transformer_object()

preprocessing_obj.fit_transform(X)

SpecificationError: nested renamer is not supported

In [134]:
X = train_df.drop('Price',axis = 1)
y = train_df['Price']

In [120]:
c = DataTransformation(data_transformation_config= DataTransformationConfig,
                        data_ingestion_artifact= DataIngestionArtifact,
                        data_validation_artifact= DataValidationArtifact)
v = c.get_data_transformer_object()
v.fit_transform(train_df,y=None)

ValueError: not enough values to unpack (expected 3, got 2)

In [96]:
c = np.array(train_df)

In [70]:
train_df.drop('Price',inplace = True,axis = 1)

In [73]:
np.array(train_df).reshape(-1,1)

array([['Jet Airways'],
       ['27/06/2019'],
       ['Mumbai'],
       ...,
       ['15h 25m'],
       ['1 stop'],
       ['In-flight meal not included']], dtype=object)

In [75]:
train_df.shape

(7478, 10)

In [157]:
! pip install pandas==0.19.2


     ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/9.2 MB 991.0 kB/s eta 0:00:10
     ---------------------------------------- 0.1/9.2 MB 1.6 MB/s eta 0:00:06
     - -------------------------------------- 0.2/9.2 MB 1.8 MB/s eta 0:00:05
     -- ------------------------------------- 0.5/9.2 MB 2.9 MB/s eta 0:00:04
     --- ------------------------------------ 0.8/9.2 MB 3.8 MB/s eta 0:00:03
     ----- ---------------------------------- 1.2/9.2 MB 4.4 MB/s eta 0:00:02
     ----- ---------------------------------- 1.2/9.2 MB 4.3 MB/s eta 0:00:02
     ----- ---------------------------------- 1.3/9.2 MB 3.6 MB/s eta 0:00:03
     ----- ---------------------------------- 1.3/9.2 MB 3.6 MB/s eta 0:00:03
     ----- ---------------------------------- 1.3/9.2 MB 2.9 MB/s eta 0:00:03
     ---------- ----------------------------- 2.4/9.2 MB 4.3 MB/s eta 0:00:02
     ---------- ----------------------------- 2.4/9.2 MB 4.3 MB/s eta

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [13 lines of output]
      f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\setuptools\__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      error in pandas setup command: 'install_requires' must be a string or list of strings containing valid project/version requirement specifiers; Expected end or semicolon (after version specifier)
          pytz >= 2011k
               ~~~~~~~^
  